# $n$-Gram language models

**Natural Language Processing 2024-2025**

|   | Team  Members             |          |                                                          |
|---|---------------------------|----------|----------------------------------------------------------|
| 1 |   Anna Chatzipapadopoulou | f3322411 | ann.chatzipapadopoul@aueb.gr / annachatzipap@gmail.com   |                   
| 2 |   Alviona Mancho          | f3322405 | alv.mantso@aueb.gr  / alviona.mantso@gmail.com           |

## Imports

In [ ]:
import nltk
import evaluate
from nltk.util import ngrams
from nltk.corpus import brown
from nltk.tokenize import TweetTokenizer
nltk.download('brown')
from Levenshtein import distance
import numpy as np
import math
import random
from collections import Counter
import re

from sklearn.model_selection import train_test_split

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


ModuleNotFoundError: No module named 'Levenshtein'

## Preliminaries

In [ ]:
class Tools:
    @staticmethod
    def format_sequence(sequence):
        # Remove all occurrences of '*start*' from the beginning of the sequence
        while sequence and sequence[0] == '*start*':
            sequence.pop(0)
        return ' '.join(sequence)

In [ ]:

class TextProcessor:
    def __init__(self):
        TextProcessor.tokenizer = TweetTokenizer()

    @staticmethod
    def tokenize_corpus(corpus):
        """
        Tokenizes the corpus by splitting it into sentences, and then tokenizing each sentence into words.
        """
        # Split the corpus into sentences
        sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', corpus)

        # Split each sentence into words
        tokenized_sentences = [TextProcessor.tokenizer.tokenize(sent) for sent in sentences]

        return tokenized_sentences

    @staticmethod
    def create_vocabulary(tokenized_corpus, occurrences_threshold=10):
        """
        Creates a vocabulary from the tokenized corpus by counting word occurrences
        and selecting words that meet or exceed the occurrences threshold.
        """
        # tokenized_corpus = TextProcessor._remove_punct(tokenized_corpus)
        word_counts = Counter(word for sentence in tokenized_corpus for word in sentence)
        vocabulary = {word for word, count in word_counts.items() if count >= occurrences_threshold}

        return list(vocabulary)

    @staticmethod
    def process_tokenized_corpus(tokenized_corpus, vocabulary):
        """
        Processes the tokenized corpus by replacing words not in the vocabulary with '*UNK*'.
        """
        # tokenized_corpus = TextProcessor._remove_punct(tokenized_corpus)
        def filter_sentence(sentence):
            return [word if word in vocabulary else '*UNK*' for word in sentence]

        # Apply the filter to each sentence in the tokenized corpus
        processed_corpus = [filter_sentence(sentence) for sentence in tokenized_corpus]

        return processed_corpus

    @staticmethod
    def introduce_noise(word, mapping, replace_prob = 0.1, delete_prob = 0.04):
        """Introduces random noise to a given word by selectively replacing or deleting characters
        based on specified probabilities."""
        noisy_word = [
            random.choice(mapping[char.lower()]).upper() if char.isupper() else random.choice(mapping[char.lower()])
            if char.lower() in mapping and random.random() < replace_prob
            else char
            for char in word if random.random() >= delete_prob
        ]
        return ''.join(noisy_word)

## $n$-gram Language Model (LM) Implementation

* #### Probabilities  (`calculate_ngram_prob`)
  $P_{bigram}(w_2|w_1) = \frac{C(w_1,w_2) + \alpha}{C(w_1) + \alpha \cdot|V|} $

  $P_{trigram}(w_3|w_1,w_2) = \frac{C(w_1,w_2,w_3) + \alpha}{C(w_1,w_2) + \alpha \cdot |V|}$

  $P_{inter}(w_3|w_1,w_2) = \lambda \cdot P(w_3|w_1,w_2) +(1-\lambda) \cdot P(w_3|w_2) $

  where

  $ C(w_1) $ : unigram count (`unigram_counter`)

  $ C(w_1,w_2) $ : bigram count (`bigram_counter`)

  $ C(w_1,w_2,w_3) $ : trigram count (`trigram_counter`)

  $ 0 \leq\alpha \leq1 $ :  smoothing hyper-parameter (`alpha`)

  $ 0 \leq\lambda \leq1 $ (`lamda`)
  |$V$|: vocabulary size (`vocabulary_size`)

* #### Autocompletion
    1. Selecting the most probable next word (`ngram_auto_complete`)
    2. Using beam search (`beam_search_autocomplete`)


* #### Spelling Correction (`beam_search_spelling_corrector`)
  $\hat{t}_1^k = \arg\max_{t_1^k} \; \lambda_1 \cdot log_2 P(t_1^k) + \lambda_2 \cdot log_2 P(w_1^k | t_1^k)$
  
  where

  $w_1^k$ : the observed sequence (noisy sentence)

  $ 0 \leq\lambda_1, \lambda_2 \leq1 $ (`lambda1`, `lambda2`)

  $ P(w_1^k \mid t_1^k) \approx \prod_{i=1}^k P(w_i \mid t_i) $ is calculated using the inverse Levenshtein distance, where $ w_1^k $ is the observed sequence (noisy sentence)




In [ ]:
class Ngram:
    def __init__(self, vocabulary, alpha = 0.01, lamda=0.9,lambda1=0.3,lambda2 = 1.0):
        self.alpha = alpha
        self.lamda = lamda
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        self.vocabulary = vocabulary
        self.vocabulary_size = len(vocabulary)
        self.unigram_counter, self.bigram_counter, self.trigram_counter = None, None, None


    def train_ngram(self, tokenized_corpus):
        unigram_counter = Counter()
        bigram_counter = Counter()
        trigram_counter = Counter()

        for sent in tokenized_corpus:
            unigram_counter.update([gram for gram in ngrams(sent, 1, pad_left=True, pad_right=True,
                                                   left_pad_symbol='*start*',right_pad_symbol='*end*') ])
            bigram_counter.update([gram for gram in ngrams(sent, 2, pad_left=True, pad_right=True,
                                                        left_pad_symbol='*start*',right_pad_symbol='*end*') ])
            trigram_counter.update([gram for gram in ngrams(sent, 3, pad_left=True, pad_right=True,
                                                        left_pad_symbol='*start*',right_pad_symbol='*end*') ])

        self.unigram_counter = unigram_counter
        self.bigram_counter = bigram_counter
        self.trigram_counter = trigram_counter

    def fine_tune(self,tokenized_sentences,alphas = np.linspace(0.001,1.0,50),lambdas = np.linspace(0.0,1.0,11)):
        best_alpha, best_lambda = None, None
        best_perplexity = float('inf')
        for alpha in alphas:
            for lamb in lambdas:
                self.alpha = alpha
                self.lamda = lamb
                HC, perplexity, _ = self.sequence_ngram_prob(tokenized_sentences)
                print(f"  Alpha: {alpha:.4f}, Lambda: {lamb:.4f}, Cross Entropy: {HC:.4f}, Perplexity: {perplexity:.4f}")

                if perplexity < best_perplexity:
                    best_perplexity = perplexity
                    best_alpha = alpha
                    best_lambda = lamb

        print("-" * 45)
        print(f"Optimal Alpha: {best_alpha:.4f}, Optimal Lambda: {best_lambda:.4f}, Best Perplexity: {best_perplexity:.4f}")
        print("-" * 45)
        self.alpha = best_alpha
        self.lamda = best_lambda
        return best_alpha,best_lambda, best_perplexity

    def fine_tune_lambdas(self,noisy_sentences,original_sentences,n,beam_width=2,lambda1_values = np.linspace(0.0,1.0,11),lambda2_values = np.linspace(0.0,1.0,11)):
        best_lambda1, best_lambda2 = None, None
        best_wer, best_cer = float('inf'), float('inf')

        for lambda1 in lambda1_values:
            for lambda2 in lambda2_values:
                corrected_corpus = []
                self.lambda1 = lambda1
                self.lambda2 = lambda2
                for noisy_sentence in noisy_sentences:
                    corrected_sentence = self.beam_search_spelling_corrector(noisy_sentence,n,beam_width)
                    corrected_corpus.append(corrected_sentence)
                wer,cer = self.evaluate_spelling_correction(corrected_corpus,original_sentences)
                print(f"  Lambda1: {lambda1:.4f}, Lambda2: {lambda2:.4f}, WER: {wer:.4f}, CER: {cer:.4f}")
                if wer < best_wer or (wer == best_wer and cer < best_cer):
                    best_wer = wer
                    best_cer = cer
                    best_lambda1 = lambda1
                    best_lambda2 = lambda2

        self.lambda1 = best_lambda1
        self.lambda2 = best_lambda2

        print("-" * 45)
        print(f"Optimal Lambda1: {best_lambda1:.4f}, Optimal Lambda2: {best_lambda2:.4f}, Best Avg WER: {best_wer:.4f}, Best Avg CER: {best_cer:.4f}")
        print("-" * 45)

        return best_lambda1, best_lambda2, best_wer,best_cer


    def calculate_ngram_prob(self, tokens, n):
        """
        Calculates the smoothed probability of an n-gram using Laplace smoothing
        """
        if len(tokens) < n:
            raise ValueError(f"{n}-gram model requires at least {n-1} words as context.")

        counters = {1: self.unigram_counter, 2: self.bigram_counter, 3: self.trigram_counter}

        # Calculate numerator and denominator
        counter_num = counters[n][tuple(tokens[:n])]
        counter_denom = counters[n-1][tuple(tokens[:n-1])]

        # Return smoothed probability
        return (counter_num + self.alpha) / (counter_denom + self.alpha * self.vocabulary_size)


    def sequence_ngram_prob(self, sentences_tokenized, n=None):
        """
        Calculates the probability and perplexity of a sequence of tokenized sentences using n-gram models.
        Interpolates between bigram and trigram models if n is not specified, using lambda as the interpolation weight.
        """
        if n == None:
            bigram_prob_coeff = 1- self.lamda
            trigram_prob_coeff = self.lamda
        elif n == 2:
            bigram_prob_coeff = 1
            trigram_prob_coeff = 0
        elif n == 3:
            bigram_prob_coeff = 0
            trigram_prob_coeff = 1
        else:
            raise ValueError("Only bigrams and trigrams and their interpolation are supported")

        sum_prob = 0
        ngram_cnt = 0

        n = 3
        start_tokens = (n-1)*['*start*']
        end_tokens = ['*end*']

        for sent in sentences_tokenized:
            sent = start_tokens + sent + end_tokens

            for i in range(len(sent)-n+1):
                tokens = sent[i:i + n]
                trigram_prob = self.calculate_ngram_prob(tokens, n=3)
                bigram_prob = self.calculate_ngram_prob(tokens[-2:], n=2)

                sum_prob += math.log2(trigram_prob_coeff * trigram_prob + bigram_prob_coeff * bigram_prob)
                ngram_cnt += 1

        HC = -sum_prob / ngram_cnt
        perpl = math.pow(2, HC)
        return HC, perpl, sum_prob


    def ngram_auto_complete(self, start_sentence, n, max_length=20):
        """
        Autocompletes a given start sentence using an n-gram model up to a specified maximum length.
        The model selects each subsequent word by choosing the one with the highest probability.
        """
        sequence = start_sentence[:]
        if len(sequence) < n - 1:
            raise ValueError(f"{n}-gram model requires at least {n - 1} words as context.")

        for _ in range(max_length):
            tokens = sequence[-(n-1):]
            most_probable = max(self.vocabulary + ['*end*'], key=lambda word: self.calculate_ngram_prob(tokens+[word], n=n))
            sequence.append(most_probable)

            if most_probable == '*end*':
                break

        return sequence

    def beam_search_autocomplete(self, initial_state, n, max_depth = 10, beam_width = 2):
        """
        Autocompletes a sentence using beam search with n-gram scoring.
        Expands each sequence by selecting the top `beam_width` candidates with the highest cumulative probability.
        """
        candidates = [(initial_state, 0.0)]
        if n != 2 and n != 3:
            raise ValueError("Only bigrams and trigrams are supported")

        for _ in range(max_depth):
            new_candidates = []
            for candidate, prob in candidates:
                if candidate[-1] == '*end*':
                    new_candidates.append((candidate, prob))
                    continue
                for next_state in self._generate_candidates(candidate):
                    new_prob = prob + math.log2(self._score_ngram(next_state,n))
                    new_candidates.append((next_state, new_prob))

            new_candidates = sorted(new_candidates, key=lambda x: x[1], reverse=True)
            candidates = new_candidates[:beam_width]

        best_sequence, best_prob = max(candidates, key=lambda x: x[1])
        return best_sequence


    def beam_search_spelling_corrector(self, w_sentence, n, beam_width=2):
        """
        Performs beam search to find the most probable correction for a noisy sentence.
        Combines n-gram probabilities and inverse Levenshtein distances.
        """
        candidates = [(['*start*']*(n-1), 0.0)]
        if n != 2 and n != 3:
            raise ValueError("Only bigrams and trigrams are supported")

        for i in range(len(w_sentence)):
            new_candidates = []
            word = w_sentence[i]  # Current noisy word in sentence
            spelling_candidates = [(word_cand, self.calculate_inverse_levenshtein(word, word_cand))
                for word_cand in self.vocabulary]

            for candidate, prob in candidates:
                for word_cand, p_w_t in spelling_candidates:
                    next_state = candidate + [word_cand]
                    p_t = self.calculate_ngram_prob(candidate[-(n-1):] + [word_cand], n=n)
                    new_prob = prob + self.lambda1*math.log2(p_t) + self.lambda2*math.log2(p_w_t)
                    new_candidates.append((next_state, new_prob))

            new_candidates = sorted(new_candidates, key=lambda x: x[1], reverse=True)
            candidates = new_candidates[:beam_width]
        best_sequence, best_prob = max(candidates, key=lambda x: x[1])
        return best_sequence[n-1:]


    def _generate_candidates(self, state):
        return [state + [next_word] for next_word in self.vocabulary + ['*end*']]

    def _score_ngram(self, state, n):
        prob = 1
        for i in range(n - 1, len(state)):
            ngram = state[i - (n - 1): i + 1]
            prob *= self.calculate_ngram_prob(ngram, n=n)
        return prob

    # Setters
    def set_lambda1(self,value):
        self.lambda1 = value

    def set_lambda2(self,value):
        self.lambda2 = value

    def set_alpha(self,value):
        self.alpha = value

    def set_lamda(self,value):
        self.lamda = value

    @staticmethod
    def calculate_inverse_levenshtein(w, t):
        weights = (1, 1, 2) # Costs insert: 1, delete: 1, replace: 2
        return 1 / (distance(w, t, weights=weights) + 1)

    @staticmethod
    def evaluate_spelling_correction(corrected_corpus,original_corpus):
        wer_metric = evaluate.load("wer")
        cer_metric = evaluate.load("cer")

        predictions = [' '.join(sentence) for sentence in corrected_corpus]
        references = [' '.join(sentence) for sentence in original_corpus]

        wer_score = wer_metric.compute(predictions=predictions,references=references)
        cer_score = cer_metric.compute(predictions=predictions,references=references)

        return wer_score,cer_score

    @staticmethod
    def evaluate_ngram_model(ngram_model, corpus, dataset_name, n=None):
        if n == None:
            print(f"Evaluating Interpolation Model on {dataset_name} Set:")
            HC, perplexity, _ = ngram_model.sequence_ngram_prob(corpus)
            print(f"  Cross Entropy: {HC:.4f}")
            print(f"  Perplexity: {perplexity:.4f}")
            print("-" * 45)
        else:
            print(f"Evaluating {n}-Gram Model on {dataset_name} Set:")
            HC, perplexity, _ = ngram_model.sequence_ngram_prob(corpus, n=n)
            print(f"  Cross Entropy: {HC:.4f}")
            print(f"  Perplexity: {perplexity:.4f}")
            print("-" * 45)

## Fetch and Prepocess Data

From the diverse categories available in the Brown Corpus, we select the 'romance' category. This dataset will be divided into a 80% training set, along with 10% for testing and 10% for validation (development).

In [ ]:
print('> Categories available in the Brown Corpus:\n', '\n '.join(brown.categories()))
tokenized_corpus = brown.sents(categories='romance')

train_corpus, temp_corpus = train_test_split(tokenized_corpus, test_size=0.2, random_state=40)
dev_corpus, test_corpus = train_test_split(temp_corpus, test_size=0.5, random_state=40)

> Categories available in the Brown Corpus:
 adventure
 belles_lettres
 editorial
 fiction
 government
 hobbies
 humor
 learned
 lore
 mystery
 news
 religion
 reviews
 romance
 science_fiction


Generate a vocabulary from the training corpus and preprocess the training, development, and test corpora using the created vocabulary

In [ ]:
textProcessor = TextProcessor()
# tokenized_corpus = TextProcessor.tokenize_corpus(corpus)

vocabulary = textProcessor.create_vocabulary(train_corpus)
processed_train_corpus = textProcessor.process_tokenized_corpus(train_corpus,vocabulary)
processed_dev_corpus = textProcessor.process_tokenized_corpus(dev_corpus,vocabulary)
processed_test_corpus = textProcessor.process_tokenized_corpus(test_corpus,vocabulary)

## Training and Evaluation of the bi-gram and tri-gram LM

* $ \text{CrossEntropy}: \quad HC_{bigrams} = -\frac{1}{N}\sum_{bigrams}{log_2(P(w_2|w_1))} $, where $N$ is the number of bigrams
* $ \text{Perplexity}: \quad P = 2^{HC_{bigrams}} $

---

* $ \text{CrossEntropy}: \quad HC_{trigrams} = -\frac{1}{N}\sum_{trigrams}{log_2(P(w_3|w_1, w_2))} $, where $N$ is the number of trigrams
* $ \text{Perplexity}: \quad P = 2^{HC_{trigrams}} $

---

* $ \text{CrossEntropy}: \quad HC_{inter} = -\frac{1}{N}\sum_{trigrams}{log_2(\lambda \cdot P(w_3|w_1,w_2) +(1-\lambda) \cdot P(w_3|w_2))} $, where $N$ is the number of trigrams
* $ \text{Perplexity}: \quad P = 2^{HC_{inter}} $

In [ ]:
ngram_model = Ngram(vocabulary)

# Train
ngram_model.train_ngram(processed_train_corpus)

print(f"Evaluating Ngram Model with alpha = {ngram_model.alpha} and lambda = {ngram_model.lamda}")
print("=" * 60)

# Evaluate train dev and test sets for bigram, trigram, and interpolated ngram
for dataset_name, corpus in [("Train", processed_train_corpus),("Dev", processed_dev_corpus), ("Test", processed_test_corpus)]:
    for n in [2, 3]:
        Ngram.evaluate_ngram_model(ngram_model, corpus, dataset_name, n)
    Ngram.evaluate_ngram_model(ngram_model, corpus, dataset_name)

Evaluating Ngram Model with alpha = 0.01 and lambda = 0.9
Evaluating 2-Gram Model on Train Set:
  Cross Entropy: 3.6767
  Perplexity: 12.7881
---------------------------------------------
Evaluating 3-Gram Model on Train Set:
  Cross Entropy: 2.8925
  Perplexity: 7.4256
---------------------------------------------
Evaluating Interpolation Model on Train Set:
  Cross Entropy: 2.9136
  Perplexity: 7.5349
---------------------------------------------
Evaluating 2-Gram Model on Dev Set:
  Cross Entropy: 4.4258
  Perplexity: 21.4937
---------------------------------------------
Evaluating 3-Gram Model on Dev Set:
  Cross Entropy: 5.2970
  Perplexity: 39.3135
---------------------------------------------
Evaluating Interpolation Model on Dev Set:
  Cross Entropy: 4.7158
  Perplexity: 26.2785
---------------------------------------------
Evaluating 2-Gram Model on Test Set:
  Cross Entropy: 4.5772
  Perplexity: 23.8713
---------------------------------------------
Evaluating 3-Gram Model on 

### Fine-Tuning the $n$-gram Model

In this section, we fine-tune the **`alpha`** and **`lambda`** hyperparameters to optimize our Ngram language model. The goal is to find values for these parameters that minimize **perplexity** on the development set.

- **Alpha (α)**: The smoothing parameter for Laplace smoothing, which adjusts for unseen words in the N-gram model.
- **Lambda (λ)**: The interpolation weight for combining bigram and trigram models. Higher `lambda` values favor the trigram model.


In [ ]:
best_alpha, best_lambda,best_perplexity = ngram_model.fine_tune(processed_dev_corpus)

  Alpha: 0.0010, Lambda: 0.0000, Cross Entropy: 4.5054, Perplexity: 22.7123
  Alpha: 0.0010, Lambda: 0.1000, Cross Entropy: 4.3073, Perplexity: 19.7979
  Alpha: 0.0010, Lambda: 0.2000, Cross Entropy: 4.2481, Perplexity: 19.0018
  Alpha: 0.0010, Lambda: 0.3000, Cross Entropy: 4.2228, Perplexity: 18.6718
  Alpha: 0.0010, Lambda: 0.4000, Cross Entropy: 4.2193, Perplexity: 18.6266
  Alpha: 0.0010, Lambda: 0.5000, Cross Entropy: 4.2347, Perplexity: 18.8263
  Alpha: 0.0010, Lambda: 0.6000, Cross Entropy: 4.2702, Perplexity: 19.2963
  Alpha: 0.0010, Lambda: 0.7000, Cross Entropy: 4.3318, Perplexity: 20.1370
  Alpha: 0.0010, Lambda: 0.8000, Cross Entropy: 4.4342, Perplexity: 21.6193
  Alpha: 0.0010, Lambda: 0.9000, Cross Entropy: 4.6250, Perplexity: 24.6753
  Alpha: 0.0010, Lambda: 1.0000, Cross Entropy: 5.5364, Perplexity: 46.4124
  Alpha: 0.0214, Lambda: 0.0000, Cross Entropy: 4.4422, Perplexity: 21.7392
  Alpha: 0.0214, Lambda: 0.1000, Cross Entropy: 4.3848, Perplexity: 20.8913
  Alpha: 0.0

### Evaluation after Finetuning

In [ ]:

print(f"Evaluating Ngram Model with alpha = {ngram_model.alpha} and lambda = {ngram_model.lamda}")
print("=" * 60)
# Evaluate train and dev and test sets for interpolated ngram
for dataset_name, corpus in [("Train", processed_train_corpus),("Dev", processed_dev_corpus), ("Test", processed_test_corpus)]:
    Ngram.evaluate_ngram_model(ngram_model, corpus, dataset_name)

Evaluating Ngram Model with alpha = 0.001 and lambda = 0.4
Evaluating Interpolation Model on Train Set:
  Cross Entropy: 2.5379
  Perplexity: 5.8076
---------------------------------------------
Evaluating Interpolation Model on Dev Set:
  Cross Entropy: 4.2193
  Perplexity: 18.6266
---------------------------------------------
Evaluating Interpolation Model on Test Set:
  Cross Entropy: 4.4071
  Perplexity: 21.2157
---------------------------------------------


## Demonstration of Autocompletion

In this part, the bi-gram and tri-gram models are utilized for autocompleting an incomplete sentence, through predicting potential next words based on the context provided by preceding words.

The bigram model considers the immediate previous word to suggest the next word, while the trigram model takes into account the two preceding words. The following examples  of generated texts confirm that the trigram
model generates more fluent texts than the bigram model.

### 1. Selecting the most probable next word

$\text{'Neither did he care that...'}$

In [ ]:
start_sentence = ['*start*', 'Neither', 'did', 'he', 'care', 'that']
best_sequence = ngram_model.ngram_auto_complete(start_sentence, n=2, max_length=10)
print('Bi-gram completion : ', Tools.format_sequence(best_sequence))

best_sequence = ngram_model.ngram_auto_complete(start_sentence, n=3, max_length=30)
print('Tri-gram completion: ', Tools.format_sequence(best_sequence))

Bi-gram completion :  Neither did he care that he had been a little more than the same time
Tri-gram completion:  Neither did he care that he had been a good thing for a couple of weeks , his mother said , `` I don't know what was so beautiful , so that they had been


$\text{'He had...'}$

In [ ]:
start_sentence = ['*start*', 'He','had']
best_sequence = ngram_model.ngram_auto_complete(start_sentence, n=2, max_length=10)
print('Bi-gram completion : ', Tools.format_sequence(best_sequence))

best_sequence = ngram_model.ngram_auto_complete(start_sentence, n=3, max_length=40)
print('Tri-gram completion: ', Tools.format_sequence(best_sequence))

Bi-gram completion :  He had been a little more than the same time , and
Tri-gram completion:  He had never been here at this hour . *end*


$\text{'She never ...'}$

In [ ]:
start_sentence = ['*start*', 'She', 'never']
best_sequence = ngram_model.ngram_auto_complete(start_sentence, n=2, max_length=10)
print('Bi-gram completion : ', Tools.format_sequence(best_sequence))

best_sequence = ngram_model.ngram_auto_complete(start_sentence, n=3, max_length=10)
print('Tri-gram completion: ', Tools.format_sequence(best_sequence))

Bi-gram completion :  She never been a little more than the same time , and
Tri-gram completion:  She never got on the other end of the tractor in time


$\text{'I am...'}$

In [ ]:
start_sentence = ['*start*', 'I', 'am']
best_sequence = ngram_model.ngram_auto_complete(start_sentence, n=2, max_length=5)
print('Bi-gram completion : ', Tools.format_sequence(best_sequence))

best_sequence = ngram_model.ngram_auto_complete(start_sentence, n=3, max_length=5)
print('Tri-gram completion: ', Tools.format_sequence(best_sequence))


Bi-gram completion :  I am innocent '' . *end*
Tri-gram completion:  I am innocent '' . *end*


### 2. Using Beam Search

$\text{'I took...'}$

In [ ]:
start_sentence = ['*start*', 'I', 'took']
best_sequence = ngram_model.beam_search_autocomplete(initial_state=start_sentence, n=2, max_depth=12, beam_width=5)
print('Bi-gram completion : ', Tools.format_sequence(best_sequence))

best_sequence = ngram_model.beam_search_autocomplete(initial_state=start_sentence, n=3, max_depth=12, beam_width=5)
print('Tri-gram completion: ', Tools.format_sequence(best_sequence))

Bi-gram completion :  I took him . *end*
Tri-gram completion:  I took off one of these days , he said . *end*


$\text{'When he thought that...'}$

In [ ]:
start_sentence =['*start*', 'When','he','thought','that']
best_sequence = ngram_model.beam_search_autocomplete(initial_state=start_sentence, n=2, max_depth=12, beam_width=5)
print('Bi-gram completion : ', Tools.format_sequence(best_sequence))

best_sequence = ngram_model.beam_search_autocomplete(initial_state=start_sentence, n=3, max_depth=12, beam_width=5)
print('Tri-gram completion: ', Tools.format_sequence(best_sequence))

Bi-gram completion :  When he thought that '' . *end*
Tri-gram completion:  When he thought that maybe Wally was right . *end*


$\text{'She...'}$

In [ ]:
start_sentence = ['*start*', 'She']
best_sequence = ngram_model.beam_search_autocomplete(initial_state=start_sentence, n=2, max_depth=12, beam_width=3)
print('Bi-gram completion : ', Tools.format_sequence(best_sequence))

best_sequence = ngram_model.beam_search_autocomplete(initial_state=start_sentence, n=3, max_depth=12, beam_width=3)
print('Tri-gram completion: ', Tools.format_sequence(best_sequence))

Bi-gram completion :  She said . *end*
Tri-gram completion:  She said , `` I don't know what he did . *end*


$\text{'She never...'}$

In [ ]:
start_sentence = ['*start*', 'She', 'never']
best_sequence = ngram_model.beam_search_autocomplete(initial_state=start_sentence, n=2, max_depth=10, beam_width=4)
print('Bi-gram completion : ', Tools.format_sequence(best_sequence))

best_sequence = ngram_model.beam_search_autocomplete(initial_state=start_sentence, n=3, max_depth=10, beam_width=4)
print('Tri-gram completion: ', Tools.format_sequence(best_sequence))

Bi-gram completion :  She never heard her . *end*
Tri-gram completion:  She never got on well with his lips . *end*


$\text{' ``That's...'}$

In [ ]:
start_sentence = ['``', 'That\'s']
best_sequence = ngram_model.beam_search_autocomplete(initial_state=start_sentence, n=2, max_depth=10, beam_width=4)
print('Bi-gram completion : ', Tools.format_sequence(best_sequence))

best_sequence = ngram_model.beam_search_autocomplete(initial_state=start_sentence, n=3, max_depth=10, beam_width=4)
print('Tri-gram completion: ', Tools.format_sequence(best_sequence))

Bi-gram completion :  `` That's what he had been a little . *end*
Tri-gram completion:  `` That's what you have a drink . *end*


In [ ]:
start_sentence = ['``', 'That\'s']
best_sequence = ngram_model.beam_search_autocomplete(initial_state=start_sentence, n=2, max_depth=10, beam_width=20)
print('Bi-gram completion : ', Tools.format_sequence(best_sequence))

best_sequence = ngram_model.beam_search_autocomplete(initial_state=start_sentence, n=3, max_depth=10, beam_width=20)
print('Tri-gram completion: ', Tools.format_sequence(best_sequence))

Bi-gram completion :  `` That's true . *end*
Tri-gram completion:  `` That's true '' , she said . *end*


$\text{'A young...'}$

In [ ]:
start_sentence = ['*start*', 'A', 'young']
best_sequence = ngram_model.beam_search_autocomplete(initial_state=start_sentence, n=2, max_depth=10, beam_width=5)
print(Tools.format_sequence(best_sequence))

best_sequence = ngram_model.beam_search_autocomplete(initial_state=start_sentence, n=3, max_depth=20, beam_width=5)
print(Tools.format_sequence(best_sequence))

A young man . *end*
A young man . *end*


## Demonstration of Context-aware Spelling Correction

$\text{'This is a reason why I thought that .'} \xrightarrow{\text{noise}} \text{'Tais ist ank etwason pbry k thught taat .'} \xrightarrow{\text{spelling corrector}} ... $

In [ ]:
test_sentence = ['Tais', 'ist', 'ank', 'etwason', 'pbry', 'k', 'thught', 'taat', '.']
correct_sent = ngram_model.beam_search_spelling_corrector(test_sentence, n=3)
print(' '.join(correct_sent))

This is a reason why I had to be


In this part, an artificial test dataset is created to evaluate the context-aware spelling corrector. The process involves the following steps:

1. **Dataset Selection**: The same test dataset used for evaluating the language models was utilized.

2. **Character Replacement**: To introduce noise into the sentences, each character is replaced with an adjacent keyboard character with a small probability. This approach simulates common typing errors, as characters that are near each other on the keyboard are more likely to be mistyped.

3. **Character Deletion**: There is another small probability of randomly deleting some characters from the sentences was introduced. This simulates realistic scenarios where characters might be inadvertently omitted during typing.

The resulting dataset retains the overall structure and context of the original sentences while incorporating typographical errors that challenge the spelling corrector.

### Replacement mapping (QWERTY keyboard)

In [ ]:
replacement_mapping = {
    'a': ['q', 'w', 's', 'z'],
    'b': ['v', 'g', 'h', 'n'],
    'c': ['x', 'd', 'f', 'v'],
    'd': ['s', 'e', 'r', 'f', 'x', 'c'],
    'e': ['w', 's', 'd', 'r'],
    'f': ['d', 'r', 't', 'g', 'c', 'v'],
    'g': ['f', 't', 'y', 'h', 'v', 'b'],
    'h': ['g', 'y', 'u', 'j', 'b', 'n'],
    'i': ['u', 'j', 'k', 'o'],
    'j': ['h', 'u', 'i', 'k', 'n', 'm'],
    'k': ['j', 'i', 'o', 'l', 'm'],
    'l': ['k', 'o', 'p'],
    'm': ['n', 'j', 'k'],
    'n': ['b', 'h', 'j', 'm'],
    'o': ['i', 'k', 'l', 'p'],
    'p': ['o', 'l'],
    'q': ['a', 'w'],
    'r': ['e', 'd', 'f', 't'],
    's': ['a', 'w', 'e', 'd', 'x', 'z'],
    't': ['r', 'f', 'g', 'y'],
    'u': ['y', 'h', 'j', 'i'],
    'v': ['c', 'f', 'g', 'b'],
    'w': ['q', 'a', 's', 'e'],
    'x': ['z', 's', 'd', 'c'],
    'y': ['t', 'g', 'h', 'u'],
    'z': ['a', 's', 'x'],
}

In [ ]:
random.seed = 50

noisy_test_corpus = [
    [TextProcessor.introduce_noise(word, replacement_mapping, 0.1, 0.05) for word in sentence]
    for sentence in test_corpus
]

### Spelling Correction of Noisy Sentences

In [ ]:
corrected_corpus = []

for noisy_sentence in noisy_test_corpus:
    corrected_sentence = ngram_model.beam_search_spelling_corrector(noisy_sentence, n=3)
    corrected_corpus.append(corrected_sentence)

In [ ]:
print("Randomly Selected Original, Noisy, and Corrected Sentences:\n")
combined_data = list(zip(test_corpus, noisy_test_corpus, corrected_corpus))
sampled_data = random.sample(combined_data, 5)

for original, noisy, corrected in sampled_data:
    print(f"Original:  {' '.join(original)}")
    print(f"Noisy:     {' '.join(noisy)}")
    print(f"Corrected: {' '.join(corrected)}")
    print()

Randomly Selected Original, Noisy, and Corrected Sentences:

Original:  We'll drop Mr. Rawlings off in Ardmore '' , Julia said , and for the merest second George was reminded of her father's tone with servants .
Noisy:     Qe'll crop Nr. Dawlint if on Srdoe '' , Hukiz aid , and for the merest sexond Yeorge wss femided of her fatnef' tone with srrvants .
Corrected: `` You are the only one of the day he was , and for a moment , her eyes , so he can be in town in

Original:  Mousie Chandler had been to school with her someplace near Baltimore and tried to explain rather than defend her to the gang having lunch at Horne's .
Noisy:     Nousiw Xhandpr had bren ro school wit hwd soeplqce near Naltimlre ad griex ro explain rqthdr than defend her to he gang havin lunh at Uorne's 
Corrected: I had had no room school it had been a tiny baby when No in rather than end her to let go past her , and I

Original:  The heart , Phil .
Noisy:     Yhe hear , Ohil .
Corrected: She had a lot .

Original:  J

### Evaluation of the Context-aware Spelling Corrector

[Word Error Rate (WER)](https://huggingface.co/spaces/evaluate-metric/wer)  $= \frac{S + D + I}{N} = \frac{S + D + I}{S + D + C}$

> This value indicates the average number of errors per reference word. The lower the value, the better the performance (0 is a perfect score).

[Character error rate (CER)](https://huggingface.co/spaces/evaluate-metric/cer) $= \frac{S + D + I}{N} = \frac{S + D + I}{S + D + C}$

> CER’s output is not always a number between 0 and 1, in particular when there is a high number of insertions. The lower the value, the better the performance (0 is a perfect score).


where

* $S$ : # of substitutions
* $D$ : # of deletions
* $I$ : # of insertions
* $C$ : # of correct words/characters
* $N$ : # of words/characters in the reference $(N=S+D+C)$

In [ ]:
print(f"Evaluating Ngram Model with lambda1 = {ngram_model.lambda1} and lambda2 = {ngram_model.lambda2}")
print("=" * 60)
wer_score , cer_score = ngram_model.evaluate_spelling_correction(corrected_corpus,test_corpus)
print(f"Word Error Rate (WER): {wer_score:.2f}")
print(f"Character Error Rate (CER): {cer_score:.2f}")

Evaluating Ngram Model with lambda1 = 0.3 and lambda2 = 1.0
Word Error Rate (WER): 0.76
Character Error Rate (CER): 0.57


### Fine-Tuning $\lambda_1$ and $\lambda_2$ for Optimal WER and CER

In this section, we fine-tune the **`lambda1`** and **`lambda2`** hyperparameters to optimize our $n$-gram language model. The goal is to find values for these parameters that minimize **wer** and **cer** on the development set.

- **Lambda1 ($\lambda_1$)**: Controls the influence of n-gram probabilities in the model.
- **Lambda2 ($\lambda_2$)**: Determines the weighting of the inverse Levenshtein distance in the beam search spelling correction.


In [ ]:
random.seed = 50

noisy_dev_corpus = [
    [TextProcessor.introduce_noise(word, replacement_mapping, 0.1, 0.05) for word in sentence]
    for sentence in dev_corpus
]

In [ ]:
ngram_model.fine_tune_lambdas(noisy_dev_corpus,dev_corpus,3)

  Lambda1: 0.0000, Lambda2: 0.0000, WER: 0.9920, CER: 0.7730
  Lambda1: 0.0000, Lambda2: 0.1000, WER: 0.4427, CER: 0.2588
  Lambda1: 0.0000, Lambda2: 0.2000, WER: 0.4427, CER: 0.2588
  Lambda1: 0.0000, Lambda2: 0.3000, WER: 0.4427, CER: 0.2588
  Lambda1: 0.0000, Lambda2: 0.4000, WER: 0.4427, CER: 0.2588
  Lambda1: 0.0000, Lambda2: 0.5000, WER: 0.4427, CER: 0.2588
  Lambda1: 0.0000, Lambda2: 0.6000, WER: 0.4427, CER: 0.2588
  Lambda1: 0.0000, Lambda2: 0.7000, WER: 0.4427, CER: 0.2588
  Lambda1: 0.0000, Lambda2: 0.8000, WER: 0.4427, CER: 0.2588
  Lambda1: 0.0000, Lambda2: 0.9000, WER: 0.4427, CER: 0.2588
  Lambda1: 0.0000, Lambda2: 1.0000, WER: 0.4427, CER: 0.2588
  Lambda1: 0.1000, Lambda2: 0.0000, WER: 0.9635, CER: 0.7448
  Lambda1: 0.1000, Lambda2: 0.1000, WER: 0.8867, CER: 0.6826
  Lambda1: 0.1000, Lambda2: 0.2000, WER: 0.8338, CER: 0.6325
  Lambda1: 0.1000, Lambda2: 0.3000, WER: 0.7940, CER: 0.5933
  Lambda1: 0.1000, Lambda2: 0.4000, WER: 0.6744, CER: 0.5072
  Lambda1: 0.1000, Lambd

(np.float64(0.1), np.float64(1.0), 0.41975308641975306, 0.2886060229742316)

In [ ]:
corrected_corpus = []
ngram_model.set_lambda1(0.1)
ngram_model.set_lambda2(1.0)
for noisy_sentence in noisy_test_corpus:
    corrected_sentence = ngram_model.beam_search_spelling_corrector(noisy_sentence, n=3,beam_width=2)
    corrected_corpus.append(corrected_sentence)

In [ ]:
print(f"Evaluating Ngram Model with lambda1 = {ngram_model.lambda1} and lambda2 = {ngram_model.lambda2}")
print("=" * 60)
wer_score , cer_score = ngram_model.evaluate_spelling_correction(corrected_corpus,test_corpus)
print(f"Word Error Rate (WER): {wer_score:.2f}")
print(f"Character Error Rate (CER): {cer_score:.2f}")

Evaluating Ngram Model with lambda1 = 0.1 and lambda2 = 1.0
Word Error Rate (WER): 0.42
Character Error Rate (CER): 0.29
